### Secuencias 

Las secuencias (`seq`) son otro tipo de colecciones que ofrece F#, y se utilizan habitualmente cuando uno tiene una gran cantidad de datos. En efecto, a diferencia de las listas, los elementos de una secuencia se computan a medida que son requeridos. Por otra parte, las secuencias se asocian al tipo ` IEnumerable<T>` del ecosistema .NET, con lo cual son adecuadas para ocasiones en las que uno tiene que interactuar con bibliotecas o interfaces escritas en otros lenguajes, como C# o VB.

Para construir una secuencia, usamos (ejem...) un constructor `for... in ...do`, como se ve en los siguientes ejemplos:

In [1]:
let l = seq { for x in 1..5 -> x }
l

[ 1, 2, 3, 4, 5 ]

In [2]:
let sq =  seq {for x in 1..5 -> x*x }
sq

[ 1, 4, 9, 16, 25 ]

El uso de la flecha `->` es un edulcorante sintáctico que se usa cuando la expresión que evalua cada elemento de la secuencia es simple. Sino, se utiliza el constructor `do...yield`

In [3]:
let even = seq {for x in 0..3..30 do
                if (x % 2 = 0) then 
                    yield x}
even                   

[ 0, 6, 12, 18, 24, 30 ]

Una diferencia importante respecto de las listas es que no es posible acceder a los elementos de una secuencia usando el índice en forma habitual, como por ejemplo `even[3]`

In [4]:
even[3]

Error: input.fsx (1,1)-(1,8) typecheck error The type 'IEnumerable<_>' does not define the field, constructor or member 'Item'.

Sino que se debe utilizar el método `Seq.item` o su equivalente `Seq.nth`:

In [5]:
Seq.item 3 even

18

In [6]:
Seq.nth 0 even

0

In [7]:
Seq.nth 14 even

Error: System.ArgumentException: The input sequence has an insufficient number of elements.
seq was short by 9 elements (Parameter 'index')
   at Microsoft.FSharp.Collections.Internal.IEnumerator.nth[T](Int32 index, IEnumerator`1 e) in D:\a\_work\1\s\src\FSharp.Core\seq.fs:line 43
   at Microsoft.FSharp.Collections.SeqModule.Item[T](Int32 index, IEnumerable`1 source) in D:\a\_work\1\s\src\FSharp.Core\seq.fs:line 613
   at <StartupCode$FSI_0012>.$FSI_0012.main@()
   at System.RuntimeMethodHandle.InvokeMethod(Object target, Void** arguments, Signature sig, Boolean isConstructor)
   at System.Reflection.MethodInvoker.Invoke(Object obj, IntPtr* args, BindingFlags invokeAttr)

Se pueden construir secuencias complejas con cualquiera de los tipos de datos de F#:

In [8]:
type Person = { Name: string; Age: int }

In [49]:
let people = seq {
    { Name = "Alice"; Age = 25 }; { Name = "Bob"; Age = 30 }; { Name = "Charlie"; Age = 35 }}
let names = seq {
                    for person in people -> person.Name
                    }
printfn "%A" names 

seq ["Alice"; "Bob"; "Charlie"]


Y usar el constructor `for` en un estilo imperativo para, por ejemplo, filtrar una secuencia: 

In [50]:
seq {for person in people do 
        if (person.Age < 33) then yield person.Name}

[ Alice, Bob ]

El constructor `yield` agrega un elemento a la secuencia, pero cuando queremos agregar varios elementos, utilizamos `yield!`. En el ejemplo siguiente, concatenamos dos secuencias en una tercera

In [51]:
let joined = 
    seq {
        yield! l
        yield! sq
    }

printfn $"l: %A{l}"
printfn $"sq: %A{sq}"
printfn $"joined: %A{joined}"

l: seq [1; 2; 3; 4; ...]
sq: seq [1; 4; 9; 16; ...]
joined: seq [1; 2; 3; 4; ...]


> Nótese el uso del descriptor de formato %A en la cadena de caracteres interpolada. 

In [52]:
let anotherJoined = 
    seq {
        yield! l
        for x in 6..10 do x 
    }

anotherJoined

[ 1, 2, 3, 4, 5, 6, 7, 8, 9, 10 ]

#### Un poco de manejo de `strings`

Empecemos a utilizar toda la potencia de las bibliotecas de .NET:

- `System.IO.Path` que maneja todo aquello que tiene que ver con _paths_, es decir, nombres completos de archivos. 
- `System.String` que contiene métodos para trabajar con `strings`

In [53]:
let filepaths = seq { "file1.txt"; "file2.exe"; "file3.txt"; "file4.doc"; "file5.pdf" }

let extensions =
    seq {
        for filepath in filepaths do
            let extension = Path.GetExtension(filepath)
            if not (String.IsNullOrEmpty(extension)) then
                yield extension.Substring(1) // Grabs substring beginning at char index 1 
    }
    |> Seq.distinct

printfn "%A" extensions 

seq ["txt"; "exe"; "doc"; "pdf"]


El método `.Substring(n)` de un objeto tipo `string` permite obtener la cadena de caracteres incluída en dicho objeto que comienza en caracter n-ésimo

In [54]:
let titulo = "La bella y graciosa moza marchóse a lavar la ropa"

printfn $"{titulo}"
printfn $"{titulo.Substring(3)}"

La bella y graciosa moza marchóse a lavar la ropa
bella y graciosa moza marchóse a lavar la ropa


#### Lazy evaluation

Como se mencionó arriba, los elementos de las secuencias se van construyendo a medida que se necesitan, en un proceso característico de muchos lenguajes funcionales que se denomina _lazy evaluation_ (evaluación ... perezosa?): 

In [12]:
let intSeq =
    seq { for n in 1 .. 10 do
            printfn "intSeq: %i" n
            yield n }
            
Seq.nth 3 intSeq
Seq.nth 5 intSeq
            

intSeq: 1
intSeq: 2
intSeq: 3
intSeq: 4
intSeq: 1
intSeq: 2
intSeq: 3
intSeq: 4
intSeq: 5
intSeq: 6


6

In [10]:
Seq.nth 5 intSeq


intSeq: 1
intSeq: 2
intSeq: 3
intSeq: 4
intSeq: 5
intSeq: 6


6

Esta característica permite que existan métodos como el siguiente, que "evalua" una secuencia infinita:

In [14]:
let allNumbers = Seq.initInfinite (fun i -> i * 2)

Sobre la cual puedo tomar una determinada cantidad de elementos con el método `Seq.take`

In [15]:
printfn "%A" (allNumbers |> Seq.take 20)
printfn "%A" (allNumbers |> Seq.take 20 |> Seq.length)
printfn "%A" (allNumbers |> Seq.take 30)
printfn "%A" (allNumbers |> Seq.take 30 |> Seq.length)

seq [0; 2; 4; 6; ...]
20
seq [0; 2; 4; 6; ...]
30


Pero hay que tener precaución al hacer este tipo de cosas:

In [16]:
printfn "%A" (allNumbers |> Seq.length) // Ojo!!!!

Error: System.InvalidOperationException: Enumeration based on System.Int32 exceeded System.Int32.MaxValue.
   at Microsoft.FSharp.Collections.Internal.IEnumerator.upto@323.System.Collections.IEnumerator.MoveNext() in D:\a\_work\1\s\src\FSharp.Core\seq.fs:line 336
   at Microsoft.FSharp.Collections.SeqModule.Length[T](IEnumerable`1 source) in D:\a\_work\1\s\src\FSharp.Core\seq.fs:line 870
   at <StartupCode$FSI_0024>.$FSI_0024.main@()
   at System.RuntimeMethodHandle.InvokeMethod(Object target, Void** arguments, Signature sig, Boolean isConstructor)
   at System.Reflection.MethodInvoker.Invoke(Object obj, IntPtr* args, BindingFlags invokeAttr)

El tipo `seq` es compatible con el tipo `IEnumerable<'T>` del ecosistema .NET. Por lo tanto, cualquier uso de una biblioteca de .NET que nos devuelva un `IEnumerable<'T>`, se usa directamente como una secuencia:

In [19]:

let cwd = System.IO.Directory.GetCurrentDirectory()
printfn "%s" cwd

let files =
        System.IO.Directory.EnumerateFiles(cwd)
        |> Seq.map(fun f -> (System.IO.Path.GetFileName(f),File.GetCreationTime(f)))
        

files 
|> Seq.iter(fun (name,date) -> printfn $"{name} created at {date}") 

files
|> Seq.toList        

/Users/flavioc/Library/Mobile Documents/com~apple~CloudDocs/Documents/Blog/es
IntroToTypes_es.ipynb created at 4/13/2023 10:15:44 AM
DiscriminatedUnions_es.ipynb created at 4/17/2023 9:39:43 AM
Fundamentals_es.ipynb created at 4/13/2023 1:51:14 PM
Results.ipynb created at 4/27/2023 2:27:32 PM
Exceptions.ipynb created at 4/25/2023 9:39:00 AM
YetAnotherTakeOnCollections.ipynb created at 5/1/2023 5:02:39 PM
IO.ipynb created at 5/2/2023 10:41:05 AM
OnCollections.ipynb created at 4/25/2023 9:39:00 AM
DUs.ipynb created at 4/17/2023 9:39:43 AM
ControlFlow.ipynb created at 4/13/2023 1:51:14 PM
Tuples.ipynb created at 4/19/2023 9:06:42 AM
MoreOnCollections.ipynb created at 4/25/2023 9:39:00 AM
Introduccion.md created at 11/3/2022 4:54:54 PM
Maps.ipynb created at 4/25/2023 9:39:00 AM
Records_es.ipynb created at 4/13/2023 10:16:13 AM
TypeExercises.ipynb created at 4/13/2023 11:11:06 AM
MasSobreFunciones.ipynb created at 4/13/2023 1:51:14 PM
Functions_es.ipynb created at 4/13/2023 1:51:14 PM
Units

index value 0 (IntroToTypes_es.ipynb, 4/13/2023 10:15:44 AM) Item1 IntroToTypes_es.ipynb Item2 2023-04-13 10:15:44Z 1 (DiscriminatedUnions_es.ipynb, 4/17/2023 9:39:43 AM) Item1 DiscriminatedUnions_es.ipynb Item2 2023-04-17 09:39:43Z 2 (Fundamentals_es.ipynb, 4/13/2023 1:51:14 PM) Item1 Fundamentals_es.ipynb Item2 2023-04-13 13:51:14Z 3 (Results.ipynb, 4/27/2023 2:27:32 PM) Item1 Results.ipynb Item2 2023-04-27 14:27:32Z 4 (Exceptions.ipynb, 4/25/2023 9:39:00 AM) Item1 Exceptions.ipynb Item2 2023-04-25 09:39:00Z 5 (YetAnotherTakeOnCollections.ipynb, 5/1/2023 5:02:39 PM) Item1 YetAnotherTakeOnCollections.ipynb Item2 2023-05-01 17:02:39Z 6 (IO.ipynb, 5/2/2023 10:41:05 AM) Item1 IO.ipynb Item2 2023-05-02 10:41:05Z 7 (OnCollections.ipynb, 4/25/2023 9:39:00 AM) Item1 OnCollections.ipynb Item2 2023-04-25 09:39:00Z 8 (DUs.ipynb, 4/17/2023 9:39:43 AM) Item1 DUs.ipynb Item2 2023-04-17 09:39:43Z 9 (ControlFlow.ipynb, 4/13/2023 1:51:14 PM) Item1 ControlFlow.ipynb Item2 2023-04-13 13:51:14Z 10 (Tuples.ipynb, 4/19/2023 9:06:42 AM) Item1 Tuples.ipynb Item2 2023-04-19 09:06:42Z 11 (MoreOnCollections.ipynb, 4/25/2023 9:39:00 AM) Item1 MoreOnCollections.ipynb Item2 2023-04-25 09:39:00Z 12 (Introduccion.md, 11/3/2022 4:54:54 PM) Item1 Introduccion.md Item2 2022-11-03 16:54:54Z 13 (Maps.ipynb, 4/25/2023 9:39:00 AM) Item1 Maps.ipynb Item2 2023-04-25 09:39:00Z 14 (Records_es.ipynb, 4/13/2023 10:16:13 AM) Item1 Records_es.ipynb Item2 2023-04-13 10:16:13Z 15 (TypeExercises.ipynb, 4/13/2023 11:11:06 AM) Item1 TypeExercises.ipynb Item2 2023-04-13 11:11:06Z 16 (MasSobreFunciones.ipynb, 4/13/2023 1:51:14 PM) Item1 MasSobreFunciones.ipynb Item2 2023-04-13 13:51:14Z 17 (Functions_es.ipynb, 4/13/2023 1:51:14 PM) Item1 Functions_es.ipynb Item2 2023-04-13 13:51:14Z 18 (Units.ipynb, 4/27/2023 2:27:32 PM) Item1 Units.ipynb Item2 2023-04-27 14:27:32Z

Aquí utilizamos los módulos de .NET

- `System.IO.Directory` que maneja todo aquello que tiene que ver con directorios, es decir, nombres completos de archivos. 
- `System.IO.File` que contiene métodos para trabajar con archivos.

y los métodos `Seq.map` y `Seq.iter` similares a los que se utilizan con listas.

### Lists comprehension

En forma similar a la de los párrafos precedentes, donde usamos la construcción `[for x in collection do ... yield expr]` para recrear secuencias; y al igual que otros lenguajes como [Python](https://www.w3schools.com/python/python_lists_comprehension.asp) o [Haskell](https://wiki.haskell.org/List_comprehension), se lo puede utilizar en F# para realizar _compresiones de listas_, esto es, una forma imperativa de construir una lista. 

In [60]:
let l = [for x in 1..5 -> x ]
l.Head

1

In [61]:
let sq =  [for x in 1..5 -> x*x ]
sq

[ 1, 4, 9, 16, 25 ]

In [62]:
let even = [for x in 0..3..30 do
                if (x % 2 = 0) then 
                    yield x]
even                     

[ 0, 6, 12, 18, 24, 30 ]

In [63]:
type Person = { Name: string; Age: int }

In [64]:
let people = [{ Name = "Alice"; Age = 25 }; { Name = "Bob"; Age = 30 }; { Name = "Charlie"; Age = 35 }]
let names = [for person in people -> person.Name]
printfn "%A" names 

["Alice"; "Bob"; "Charlie"]


In [65]:
[for person in people do 
    if (person.Age < 33) then yield person.Name]

[ Alice, Bob ]

Nótese que las listas no se evalúan en forma perezosa:

In [66]:
let intLst =
    [ for a in 1 .. 10 do
        printfn "intLst: %i" a
        yield a ]

intLst: 1
intLst: 2
intLst: 3
intLst: 4
intLst: 5
intLst: 6
intLst: 7
intLst: 8
intLst: 9
intLst: 10
